<a href="https://colab.research.google.com/github/elibtronic/whisper_research/blob/main/Whisper_and_G_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whisper G Drive Connector

This notebook will help you to run the Whisper transciption suite on a collection of files in your Google Drive. In the cell below put in your path to the folder that holds your audio.

Then hit the run button for the cell

In [68]:
# @title Whisper G Drive Connector

audio_directory = "Whisper_Files" # @param {type:"string"}
# Connect GDrive & select folder


apath = "/content/gdrive/MyDrive/"+audio_directory+"/"

try:
  print("Found the following files")
  for file in os.listdir(apath):
    print(file)
except:
  print("Could not find audio directory")



Found the following files
jfk.wav
jfk_changed.wav
jfk_changed.wav_transcription.txt


## Libraries & G Drive Connection

Run the following cell to import the python libraries we are going to use and to mount your Google Drive. You will be presented with a pop-up that will ask for your authorization to connect to your drive.

In [ ]:

#install pieces if needed
try:
    import ffmpeg
except:
    !pip install ffmpeg-python
    import ffmpeg
try:
    from whispercpp import Whisper
except:
    !pip install git+https://github.com/stlukey/whispercpp.py
    from whispercpp import Whisper


import sys
import os
import glob
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

## Prep by Transcoding

The following cell witll transcode all of your audio files into the bitrate required by whisper for transciption to work. (16000)

*Note:* This is done with a System call as bare metal `ffmpeg` is faster/better than the Python wrapper.

In [65]:
#Reset files
#!rm $apath/*_changed.wav
#!rm $apath/*.txt
#!ls $apath


#Transcode all files in the directory

for file in os.listdir(apath):
    fpath = apath + file
    trans_file = apath + file.split(".")[0]+"_changed.wav"
    print(trans_file)
    !ffmpeg -hide_banner -i $fpath  -ar 16k $trans_file


/content/gdrive/MyDrive/Whisper_Files/jfk_changed.wav
Guessed Channel Layout for Input Stream #0.0 : mono
Input #0, wav, from '/content/gdrive/MyDrive/Whisper_Files/jfk.wav':
  Metadata:
    encoder         : Lavf59.27.100
  Duration: 00:00:11.00, bitrate: 256 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 16000 Hz, mono, s16, 256 kb/s
Stream mapping:
  Stream #0:0 -> #0:0 (pcm_s16le (native) -> pcm_s16le (native))
Press [q] to stop, [?] for help
Output #0, wav, to '/content/gdrive/MyDrive/Whisper_Files/jfk_changed.wav':
  Metadata:
    ISFT            : Lavf58.76.100
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 16000 Hz, mono, s16, 256 kb/s
    Metadata:
      encoder         : Lavc58.134.100 pcm_s16le
size=     344kB time=00:00:10.88 bitrate= 258.9kbits/s speed=3.18e+03x    
video:0kB audio:344kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.022159%



## Transcribe with Whisper

The next cell will run all of your transcoded files through Whisper and create text files with the transcipts.

**NB: This might take awhile depending on how many files you are working with!**

(Who knows, I might make this system calls as well)

In [66]:
#Run Whisper on the transcribed files
transcribed_files = glob.glob(apath+"*_changed.wav")
for file in transcribed_files:
  w = Whisper('tiny')
  transcript_file = file+"_transcription.txt"
  result = w.transcribe(file)
  text = w.extract_text(result)
  final_transcipt = open(transcript_file,"w")
  final_transcipt.writelines(" ".join(text))
  final_transcipt.close()

Loading data..
Transcribing..
Extracting text...


## Done!

In your directory of files you should see a series of text files with the transcipts. This might take a bit to sync back to your Google Drive interface. You can run the cell below to see a list of all of the files that are now in your directory.

In [70]:
!ls -lh $apath

total 689K
-rw------- 1 root root 344K Jan 11 20:39 jfk_changed.wav
-rw------- 1 root root  106 Jan 11 20:40 jfk_changed.wav_transcription.txt
-rw------- 1 root root 344K Dec 21 17:51 jfk.wav
